MapReduce vs. Spark (https://www.xplenty.com/blog/apache-spark-vs-hadoop-mapreduce/)

Why do we always repeat work_count example?

The first reason is because it is the easiest one example to understand the logic behind hadoop for rookies in this area lol. 
The second reason is that this example has huge amount of applications.
The most straightforward example is that it helps companies like Amazon to calculate which items are the most frequently bought ones by customers. 
We can also figure out the most frequent doubleton or tripleton items. I believe everyone heard of the beer and diaper case, right? That is an easy task for Hadoop system.

If you really want to dig deeper, MapReduce/Spark also can do a lot of amazing things, like Matrix Multiplication. (example http://www.mathcs.emory.edu/~cheung/Courses/554/Syllabus/9-parallel/matrix-mult.html)




In [1]:
import shutil
import os 

os.environ['HADOOP_HOME'] = os.path.abspath(os.path.join(os.getcwd(), os.pardir)).replace('\\', '/')+'/util/hadoop-2.7.1'

In [2]:
import pyspark
from pyspark import SparkConf
from pyspark.context import SparkContext

sc = None 
# count = 0
while sc is None: 
    try: 
        sc = SparkContext.getOrCreate(SparkConf().setMaster("local"))
    except: 
        pass
        # print(f"tried {count} time", end='\r')
        # count += 1

# Starting off by reading the input file as text_file
text_file = sc.textFile("input.txt")
text_file

input.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

## explanation: 
### step1:
```bash
flatMap(lambda line: line.split(" "))
```
we need to break the file into lines for which splitting on the base of “ ” is required.

Here **lambda** function is an anonymous function. Sometimes we use anonymous function to make the code tighter.

It is exactly the same as:
```
def split_word(line):
    return line.split(" ")
    
flatMap(split_word)
```
### step2:
```
map(lambda word: (word, 1))
```
map function has the same use like the MapReduce **Mapper** function. 

It counts each word 1 time every time it appears. 
```
Like [I I love hadoop] --> (I,1) (I,1) (love,1) (hadoop,1)
```
Remember: It won't do things like (I,2) because we define **lambda word: (word, 1)** in the map function

### Step3:
```
reduceByKey(lambda a, b: a + b)
```
Since this function works by **key**, a and b here are the values of previous step.

For example, 
```
input for this step: (I,1) (I,1) (love,1) (hadoop,1)
output for this step: (I,2) (love,1) (hadoop,1)
```

In [3]:
counts = text_file.flatMap(lambda line: line.split(" "))
counts = counts.map(lambda word: (word, 1))
counts = counts.reduceByKey(lambda a, b: a + b)

shutil.rmtree('output', ignore_errors=True)

### output file must not exist!!! Otherwise, it will appear error.
counts.saveAsTextFile("./output")

In [4]:
'''
Here is the same map reduce scripts without any lambda function
'''

def split_word(line):
    return line.split(" ")

def word_ct(word):
    return (word,1)

def value_add(a,b):
    return a+b

counts = text_file.flatMap(split_word) \
             .map(word_ct) \
             .reduceByKey(value_add)
             
shutil.rmtree('output1', ignore_errors=True)

### output file must not exist!!! Otherwise, it will appear error.
counts.saveAsTextFile("output1/")